## Multi-Class Logistic Regression Model for Cell Annotation
#### Using data from a multiple sclerosis dataset as an example

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import scanpy as sc
import time

In [8]:
# Import data
adata_train = sc.read('/Users/aidenmomtaz/Downloads/Beng199_Projects/PyTorchProject/msPracticeData/filtered_ms_adata.h5ad')
adata_test = sc.read('/Users/aidenmomtaz/Downloads/Beng199_Projects/PyTorchProject/msPracticeData/c_data.h5ad')
train_celltype_column = "Factor Value[inferred cell type - authors labels]"
test_celltype_column = "Factor Value[inferred cell type - authors labels]"
print('data imported')

data imported


In [10]:
# Device configs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 50
learning_rate = 0.001
input_size = len(adata_train.var_names)
hidden_size = 200
output_size = len(adata_train.obs[train_celltype_column].unique())
batch_size = 200

In [11]:
# Transform
class adataDataset(Dataset):
    def __init__(self,data,labels):
        super(adataDataset,self).__init__()
        data = data.X.toarray()
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(LabelEncoder().fit_transform(labels), dtype=torch.long)
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
    
train_data = adataDataset(adata_train,adata_train.obs[train_celltype_column].values)
test_data = adataDataset(adata_test,adata_test.obs[test_celltype_column].values)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [12]:
# Define model
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(NeuralNet,self).__init__()
        self.linear1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size,output_size)

    def forward(self,x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        return out
    
model = NeuralNet(input_size,hidden_size,output_size)
if torch.cuda.device_count() > 1:
    device_indices = [0,1,2]
    model = nn.DataParallel(model, device_ids=device_indices)
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.0001)

In [13]:
# Training loop
steps_in_epoch = len(train_loader)
since = time.time()
for epoch in range(num_epochs):
    for i, (counts, labels) in enumerate(train_loader):
        counts = counts.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(counts)
        loss = criterion(outputs, labels)

        # backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%20 == 0:
            time_elapsed = time.time()-since
            print(f'Epoch: {epoch+1} Step: {i+1}/{steps_in_epoch} Loss: {loss.item():.3f} Time: {time_elapsed//60}m {time_elapsed%60:.1f}s')

Epoch: 1 Step: 20/68 Loss: 0.778 Time: 0.0m 0.1s
Epoch: 1 Step: 40/68 Loss: 0.380 Time: 0.0m 0.2s
Epoch: 1 Step: 60/68 Loss: 0.497 Time: 0.0m 0.3s
Epoch: 2 Step: 20/68 Loss: 0.218 Time: 0.0m 0.4s
Epoch: 2 Step: 40/68 Loss: 0.213 Time: 0.0m 0.5s
Epoch: 2 Step: 60/68 Loss: 0.212 Time: 0.0m 0.6s
Epoch: 3 Step: 20/68 Loss: 0.141 Time: 0.0m 0.8s
Epoch: 3 Step: 40/68 Loss: 0.087 Time: 0.0m 0.9s
Epoch: 3 Step: 60/68 Loss: 0.092 Time: 0.0m 1.0s
Epoch: 4 Step: 20/68 Loss: 0.073 Time: 0.0m 1.1s
Epoch: 4 Step: 40/68 Loss: 0.085 Time: 0.0m 1.2s
Epoch: 4 Step: 60/68 Loss: 0.085 Time: 0.0m 1.3s
Epoch: 5 Step: 20/68 Loss: 0.048 Time: 0.0m 1.4s
Epoch: 5 Step: 40/68 Loss: 0.036 Time: 0.0m 1.5s
Epoch: 5 Step: 60/68 Loss: 0.045 Time: 0.0m 1.6s
Epoch: 6 Step: 20/68 Loss: 0.021 Time: 0.0m 1.7s
Epoch: 6 Step: 40/68 Loss: 0.024 Time: 0.0m 1.8s
Epoch: 6 Step: 60/68 Loss: 0.024 Time: 0.0m 1.9s
Epoch: 7 Step: 20/68 Loss: 0.016 Time: 0.0m 2.0s
Epoch: 7 Step: 40/68 Loss: 0.016 Time: 0.0m 2.1s
Epoch: 7 Step: 60/68

In [14]:
# Test
with torch.no_grad():
    num_correct = 0
    num_samples = 0
    for counts, labels in test_loader:
        counts = counts.to(device)
        labels = labels.to(device)
        outputs = model(counts)

        _, predictions = torch.max(outputs,1)
        num_correct += (predictions == labels).sum().item()
        num_samples += labels.shape[0]

    print(f'Acc: {num_correct/num_samples}')

Acc: 0.77371239163692
